# Leer y escribir audio en Python

**Curso**: CC5213 - Recuperación de Información Multimedia  
**Profesor**: Juan Manuel Barrios  
**Fecha**: 15 de abril de 2025


## Paso previo: Instalar FFmpeg

En este ejemplo, convertiremos un audio desde su formato codificado (como un mp3, ogg, m4a) a un formato descomprimido "crudo" que no requiere ninguna librería para leerlo. El software más usado para codificar/decodificar audios y videos es **FFmpeg** https://ffmpeg.org/

Para instalar FFmpeg en Linux basta con `apt install ffmpeg`.  

Para instalar FFmpeg en Windows, entrar a https://www.gyan.dev/ffmpeg/builds/ descargar `ffmpeg-release-full.7z`, abrirlo, entrar a la carpeta `bin`, tomar el archivo `ffmpeg.exe` y dejarlo en la misma carpeta en que está este ipynb.

## Decodificando el Audio

Un audio decodificado es un array de números que representan el muestreo de la amplitud de la señal acústica a una tasa de tiempo fija.

Este muestreo de la señal de audio requiere definir dos atributos claves:

 * **Profundidad** (bit depth). Cuántos bits se ocuparán para representar cada sample (o muestra de la amplitud de la señal). Comúnmente se usa profundidad `s16le` (signed 16-bits little-endian), es decir, cada sample es un número entero de 2 bytes que va entre -32.768 y 32.767. 

 * **Tasa de muestreo** (sample rate). Cantidad de samples en un segundo (cuantas veces se muestrea la amplitud de la señal en un segundo). Un valor comúnmente usado es `44100` samples/segundo. Se puede bajar a `22050`, `11025`, o incluso a `8192` para voz. 


Nota: El oído humano no distingue mayor calidad que `44100` samples/segundo `s16le`. Sin embargo, para edición de audio se puede aumentar la calidad, por ejemplo a `96000` samples/segundo y/o a profundidad `f32le` (float 32-bits) para hacer filtros, mezclas, ediciones, pero el resultado final se vuelve a guardar a `44100`-`s16le` para reproducir.


## Crear y leer archivos .raw:

Para decodificar un audio y obtener los samples en formato crudo ("`.raw`") usar el siguiente comando:

```
ffmpeg -i video.mp4 -ac 1 -ar 8192 -acodec pcm_s16le -f s16le audio.raw
```

 * `-i video.mp4` dice que se deber leer video.mp4 como entrada (input).    
 * `-ac 1` dice que se desea crear solo un canal de salida (mono).  
 * `-ar 8192` dice que la salida debe tener 8k samples por segundo.   
 * `-acodec pcm_s16le` dice que la salida debe tener un audio pcm_s16le (codificación)    
 * `-f s16le` dice que el archivo de salida (container) deben ser los bytes s16le. Este parámetro es redundante porque es el formato por defecto para el codec pcm_s16le. Otros formatos: https://trac.ffmpeg.org/wiki/audio%20types
 * `audio.raw` es el nombre del archivo de salida.

Este comando crea el archivo `audio.raw` con los bytes de los samples con profundidad `s16le` y con una tasa de sampleo de 8192.

El archivo `audio.raw` se pueden leer con python leyendo los bytes del archivo en un array de enteros 16 bits con:
```
samples = numpy.fromfile("audio.raw", dtype=numpy.int16)
```

Para reproducir los samples en memoria se puede usar un mini-reproductor que recibe el array de samples y la tasa de sampleo:
```
    import IPython.display as ipd    
    ipd.Audio(samples, rate=8192)
```

Como el audio es un array de números, se puede operar con ellos normalmente (multiplicarlos, sumarlos, etc.). Solo se debe tener cuidado de no superar el límite numérico de la profundidad usada (-32.768 y 32.767 para 16 bits).

Para volver a guardarlos en un archivo `.raw` usar la función `tofile()` de numpy:
```
samples.tofile("audio.raw")
```


### Formato .raw versus .wav:

 * El formato `.raw` solo contiene los bytes de los samples, por lo que basta cargar el archivo a memoria y no se necesita ninguna librería o codec. Sin emabrgo, la dificultad es que se debe guardar en alguna parte la tasa de sampleo y la profundidad que se usó para crear el archivo (por ejemplo, el nombre del archivo podria contener el sample_rate y la profundidad). 

 * El formato `.wav` es esencialmente el mismo formato `.raw` pero que incluye un encabezado donde guarda el sample_rate y el bit depth (entre otros metadatos). Para poder leer un archivo `.wav` es necesario usar una librería que soporte el formato (la más común se llama `libsndfile`).


### Reproducir un archivo .raw:

 * **Opción 1**: Instalar VideoLAN Player desde https://www.videolan.org/vlc usarlo con: `vlc --demux=rawaud --rawaud-channels 1 --rawaud-samplerate 8192 --rawaud-fourcc=s16l audio.raw`
 
 * **Opción 2**: Usar el reproductor `ffplay` que viene junto con ffmpeg: `ffplay -f s16le -acodec pcm_s16le -ar 8192 audio.raw`
 
 * **Opción 3**: Usar `ffmpeg` para convertir el audio raw a otro formato (como .mp3) y usar cualquier reproductor: `ffmpeg -f s16le -acodec pcm_s16le -ar 8192 -i audio.raw audio.mp3`





## Funciones para crear y leer audio raw


In [ ]:
import numpy
import os.path
import subprocess

#funcion que recibe un nombre de archivo y llama a FFmpeg para crear un archivo raw
def convertir_audio_raw(filename, sample_rate, carpeta_temporal):
    file_raw = "{}/{}.{}.raw".format(carpeta_temporal, os.path.basename(filename), sample_rate) 
    if os.path.isfile(file_raw):
        return file_raw
    os.makedirs(carpeta_temporal, exist_ok=True)
    print("convertir {} a {} audio raw de samplerate {}".format(filename, file_raw, sample_rate))
    comando = ["ffmpeg", "-i", filename, "-ac", "1", "-ar", str(sample_rate),
                "-acodec", "pcm_s16le", "-f", "s16le",  file_raw]
    print("  COMANDO: {}".format(" ".join(comando)))
    code = subprocess.call(comando)
    if code != 0:
        raise Exception("ERROR en comando: " + " ".join(comando))
    return file_raw

#funcion que recibe un archivo raw y llama a vlc para reproducirlo
def reproducir_audio_raw(file_raw, sample_rate):
    comando = ["vlc", "--demux=rawaud", "--rawaud-channels", "1", "--rawaud-samplerate", 
               str(sample_rate), "--rawaud-fourcc=s16l", file_raw]
    print("  COMANDO: {}".format(" ".join(comando)))
    code = subprocess.call(comando)
    if code != 0:
        raise Exception("ERROR en comando: " + " ".join(comando))

print("ok")

## 1-Convertir un archivo de audio a .raw

Dado un archivo `vivaldi.mp3` (un audio de 1 minuto y 28 segundos) lo convierte a `.raw` y luego lo reproduce con VLC.


In [ ]:
# donde crear archivos intermedios
carpeta_temporal = "audios_temporales"

file_vivaldi = "vivaldi.mp3"
samplerate_vivaldi = 22050

file_raw_vivaldi = convertir_audio_raw(file_vivaldi, samplerate_vivaldi, carpeta_temporal)

print()
print("leyendo archivo \"{}\"".format(file_raw_vivaldi))
samples_vivaldi = numpy.fromfile(file_raw_vivaldi, dtype=numpy.int16)

print()
print("bytes = {} bytes".format(os.path.getsize(file_raw_vivaldi)))
print("largo = {} samples  (es igual a bytes/2)".format(len(samples_vivaldi)))
print("tiempo = {:.1f} segundos   (es igual a samples/sample_rate)".format( len(samples_vivaldi) / samplerate_vivaldi))

print()
print("primeros samples = ", samples_vivaldi)

print()
print("min sample = {}".format( min(samples_vivaldi) ))
print("max sample = {}".format( max(samples_vivaldi) ))

print()
print("reproducir \"{}\" cono VLC...".format(file_raw_vivaldi))

reproducir_audio_raw(file_raw_vivaldi, samplerate_vivaldi)


## 2-Convertir otro archivo de audio a raw

Convierte y reproduce en VLC el archivo `jaivas.mp3` de 43 segundos.


In [ ]:
file_jaivas = "jaivas.mp3"
samplerate_jaivas = 22050

file_raw_jaivas = convertir_audio_raw(file_jaivas, samplerate_jaivas, carpeta_temporal)

print()
print("leyendo {}".format(file_raw_jaivas))
samples_jaivas = numpy.fromfile(file_raw_jaivas, dtype=numpy.int16);

print()
print("bytes = {} bytes".format(os.path.getsize(file_raw_jaivas)))
print("largo = {} samples  (es igual a bytes/2)".format(len(samples_jaivas)))
print("tiempo = {:.1f} segundos   (es igual a samples/sample_rate)".format( len(samples_jaivas) / samplerate_jaivas))

print()
print("primeros samples = ", samples_jaivas)

print()
print("min sample = {}".format( min(samples_jaivas) ))
print("max sample = {}".format( max(samples_jaivas) ))

print()
print("reproducir \"{}\" cono VLC...".format(file_raw_jaivas))

reproducir_audio_raw(file_raw_jaivas, samplerate_jaivas)


## 3-Graficando ambos audios


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

#funcion que imprime un array de samples
def mostrar_samples(samples_array, titulo):
    plt.figure(figsize=(25,5))
    plt.title(titulo, {"fontsize":32})
    plt.plot(samples_array)
    plt.xlabel('Samples')
    plt.ylabel('Amplitud')
    plt.ylim(top=32767, bottom=-32768)
    plt.yticks(numpy.arange(-30000, 30001, step=10000))
    plt.margins(0)
    plt.show()

mostrar_samples(samples_jaivas, "Vivaldi")

mostrar_samples(samples_vivaldi, "Jaivas")


## 4-Sumar ambas ondas de audio

Ambos audios deben tener el mismo sample rate. Se debe tener cuidado de usar saturación para no superar los límites numéricos de 16 bits (-32768 a 32767).  

El resultado se guarda en un archivo `.raw` y se reproduce con VLC.


In [ ]:
def sumar_samples(samples1, samples2):
    largo = min(len(samples1), len(samples2))
    nuevos_samples = numpy.zeros(largo, dtype=numpy.int16) 
    for i in range(largo):
        #sample audio 1
        val1 = int(samples1[i])
        #sample audio 2
        val2 = int(samples2[i])
        # sumar ambos
        val = int((val1 + val2))
        # saturacion
        if val > 32767:
            val = 32767
        elif val < -32768:
            val = -32768
        # guardar el nuevo valor
        nuevos_samples[i] = val 
    return nuevos_samples

nuevos_samples = sumar_samples(samples_vivaldi, samples_jaivas)

nuevo_archivo_raw = "{}/suma.{}.raw".format(carpeta_temporal, samplerate_vivaldi)

print("guardando archivo {}".format(nuevo_archivo_raw))
nuevos_samples.tofile(nuevo_archivo_raw)

mostrar_samples(nuevos_samples, "Vivaldi+Jaivas")

print("reproduciendo {}".format(nuevo_archivo_raw))
reproducir_audio_raw(nuevo_archivo_raw, samplerate_vivaldi)


## 5-Efecto de eco (repetir el mismo audio con cierto desfase)

Se copia la misma onda con un leve desfase. 

Para reproducir el resultado se usará el reproductor interno de Jupyter en vez de VLC.

El reproductos interno carga los bytes del audio en el ipynb, por lo que hay que evitar usar audios muy largos.

El reproductor interno espera un array de float entre -1 y 1. Por defecto puede recibir cualquier valor porque normaliza la amplitud a `sample máximo=1` y `sample mínimo=-1`. Esto a veces no es deseable ya que no permite manipular el volumen global del audio.

Para poder ajustar el volumen global se usa `normalize=False` y se escalan los valores antes.

In [ ]:
import IPython.display as ipd    
    
def hacer_eco(samples, samplerate, segundos_desfase):
    samples_desfase = int(samplerate * segundos_desfase)
    nuevos_samples = numpy.zeros(len(samples), dtype=numpy.int16) 
    for i in range(samples_desfase, len(samples)):
        #sample actual
        val1 = int(samples[i])
        #sample anterior
        val2 = int(samples[i - samples_desfase])
        # sumar ambos, menor ponderacion al eco
        val = int((val1 + 0.8 * val2))
        # saturacion
        if val > 32767:
            val = 32767
        elif val < -32768:
            val = -32768
        # guardar el nuevo valor
        nuevos_samples[i] = val 
    return nuevos_samples

samples_eco = hacer_eco(samples_jaivas, samplerate_jaivas, 0.9)

# display.Audio() espera samples con valores entre -1 y 1.
# Con normalize=False no se hace la normalización, pero se deben escalar a mano
volumen_global = 0.5
ipd.Audio(volumen_global * (samples_eco / 32768), rate=samplerate_jaivas, autoplay=True, normalize=False)
